<a href="https://colab.research.google.com/github/DRodriguez615/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [1]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2019-11-07 16:14:54--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [                 <=>]  21.67M   802KB/s    in 28s     

2019-11-07 16:15:22 (790 KB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22727580]



In [2]:
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [128]:
import pandas as pd
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1, skipfooter=2, engine='python')
print(df.columns)
df.head()

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term'],
      dtype='object', length=144)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,35000,35000,35000.0,36 months,14.47%,1204.23,C,C2,Staff Physician,8 years,MORTGAGE,360000.0,Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,336xx,FL,19.90,0,Apr-1995,1,NaN,NaN,24,0,57259,43.2%,51,w,0.00,0.00,38187.046837,38187.05,...,30.8,0,0,1222051,169286,124600,258401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,5000,5000,5000.0,36 months,22.35%,191.86,D,D5,Director of Sales,10+ years,MORTGAGE,72000.0,Source Verified,Dec-2018,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,333xx,FL,20.12,0,Mar-2010,0,NaN,NaN,13,0,11720,47.1%,26,f,0.00,0.00,5615.977674,5615.98,...,50.0,0,0,218686,34418,18200,37786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,10000,10000,10000.0,60 months,23.40%,284.21,E,E1,NaN,< 1 year,RENT,55000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,902xx,CA,13.51,0,Apr-2007,0,44.0,88.0,9,1,11859,53.9%,11,w,9131.55,9131.55,2538.390000,2538.39,...,100.0,1,0,34386,21235,10500,12386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,17100,17100,17100.0,36 months,18.94%,626.30,D,D2,Receptionist,10+ years,RENT,38000.0,Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,150xx,PA,38.09,0,Mar-1998,1,47.0,NaN,14,0,15323,53%,21,w,13682.21,13682.21,5609.710000,5609.71,...,75.0,0,0,70954,43351,16600,41784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4000,4000,4000.0,36 months,10.72%,130.43,B,B2,Extrusion assistant,10+ years,MORTGAGE,56000.0,Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,301xx,GA,31.03,0,Sep-2006,0,NaN,NaN,7,0,4518,28.6%,11,w,3116.62,3116.62,1160.780000,1160.78,...,0.0,0,0,221310,71375,12300,77865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print(df.shape)

(128412, 144)


In [5]:
df['term'].dtype

dtype('O')

In [6]:
df.loc[0, 'term']

' 36 months'

In [7]:
term0=df.loc[0, 'term']
print(term0)

 36 months


In [8]:
term0.strip().strip(' months')

'36'

In [9]:
type(int(term0.strip().strip(' months')))

int

In [0]:
def term_2_int(mystring):
  return int(mystring.strip().strip(' months'))

In [11]:
term_2_int(term0)

36

In [12]:
df['term_int']=df['term'].apply(term_2_int)
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term', 'term_int'],
      dtype='object', length=145)

In [13]:
df[['term', 'term_int']].head()
# new term column is now converted to int form str

,term,term_int
0,36 months,36
1,36 months,36
2,60 months,60
3,36 months,36
4,36 months,36


In [14]:
df['term_double']=df['term_int']*2
df[['term', 'term_int', 'term_double']].sample(5)
# shows that new term_int column is int by being apply to apply math to it

,term,term_int,term_double
74927,36 months,36,72
79882,36 months,36,72
21637,36 months,36,72
56901,36 months,36,72
42145,36 months,36,72


Make a column named loan_status_is_great. It should contain the integer 1 if loan_status is "Current" or "Fully Paid." Else it should contain the integer 0.

In [15]:
df['loan_status']

0          Fully Paid
1          Fully Paid
2             Current
3             Current
4             Current
             ...     
128407    Charged Off
128408        Current
128409        Current
128410        Current
128411        Current
Name: loan_status, Length: 128412, dtype: object

In [16]:
df['loan_status_is_great']=df['loan_status']
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'debt_settlement_flag', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term', 'term_int', 'term_double',
       'loan_status_is_great'],
      dtype='object', length=147)

In [17]:
df[['loan_status', 'loan_status_is_great']].head()

,loan_status,loan_status_is_great
0,Fully Paid,Fully Paid
1,Fully Paid,Fully Paid
2,Current,Current
3,Current,Current
4,Current,Current


In [18]:
df['loan_status'].isnull().sum()


0

In [0]:

def loan_status_is_great_2(status):
  a = ['Fully Paid', 'Current']
  for i in a:
    print(1)
  else:
    print(0)


In [0]:
df['loan_status_is_great']=df['loan_status_is_great'].apply(loan_status_is_great_2)


In [127]:
df[['loan_status', 'loan_status_is_great']].sample(6)
# trying to figure out why this returns none instead or (1, or 0)

,loan_status,loan_status_is_great
72263,Current,1
124432,Current,1
33844,Current,1
7604,Fully Paid,1
87260,Current,1
75761,Current,1


In [123]:
# im sure there is an easier way but this is what i could come up with
# 
df['loan_status_is_great'] = df['loan_status'].map({'Current':1, 'Fully Paid': 1,
                                                    'Charged Off':0,
                                                    'Late (31-120 days)':0,
                                                    'In Grace Period':0,
                                                    'Late (16-30 days)':0, 
                                                    'Default':0})
df[['loan_status', 'loan_status_is_great']].sample(10)

,loan_status,loan_status_is_great
122872,Current,1
77005,Current,1
20951,Current,1
78838,Late (31-120 days),0
78577,Current,1
86343,Fully Paid,1
95202,In Grace Period,0
49450,Current,1
91953,Current,1
84205,Current,1


In [124]:
df['loan_status_is_great'].isnull().sum()

0

In [129]:
# Make last_pymnt_d_month and last_pymnt_d_year columns.
# first find which columns are dates?
for col in df.columns:
  if (col.endswith('_date')) or (col.endswith('_d')):
    print(col)

issue_d
last_pymnt_d
next_pymnt_d
last_credit_pull_d
hardship_start_date
hardship_end_date
payment_plan_start_date
debt_settlement_flag_date
settlement_date


In [130]:
# need last_pymnt_d column
df['last_pymnt_d'].dtype

dtype('O')

In [131]:
df.loc[0:10, 'last_pymnt_d']

0     Aug-2019
1     Jul-2019
2     Sep-2019
3     Sep-2019
4     Sep-2019
5     Sep-2019
6     Sep-2019
7     Sep-2019
8     Sep-2019
9     Oct-2019
10    Sep-2019
Name: last_pymnt_d, dtype: object

In [132]:
# need to change str to dates
df['last_pymnt_d_clean']=pd.to_datetime(df['last_pymnt_d'], infer_datetime_format=True)

df[['last_pymnt_d', 'last_pymnt_d_clean']].head()

,last_pymnt_d,last_pymnt_d_clean
0,Aug-2019,2019-08-01
1,Jul-2019,2019-07-01
2,Sep-2019,2019-09-01
3,Sep-2019,2019-09-01
4,Sep-2019,2019-09-01


In [133]:
df['last_pymnt_d_clean'].dtype

dtype('<M8[ns]')

In [135]:
df['last_pymnt_d_year']=df['last_pymnt_d_clean'].dt.year

df[['last_pymnt_d', 'last_pymnt_d_year']].head()

,last_pymnt_d,last_pymnt_d_year
0,Aug-2019,2019.0
1,Jul-2019,2019.0
2,Sep-2019,2019.0
3,Sep-2019,2019.0
4,Sep-2019,2019.0


In [136]:
df['last_pymnt_d_month']=df['last_pymnt_d_clean'].dt.month

df[['last_pymnt_d', 'last_pymnt_d_month']].head()

,last_pymnt_d,last_pymnt_d_month
0,Aug-2019,8.0
1,Jul-2019,7.0
2,Sep-2019,9.0
3,Sep-2019,9.0
4,Sep-2019,9.0


# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01

In [0]:
df['emp_title'].value_counts(ascending=False, dropna=False).head(20)

NaN                   20947
Teacher                2090
Manager                1773
Registered Nurse        952
Driver                  924
RN                      726
Supervisor              697
Sales                   580
Project Manager         526
General Manager         523
Office Manager          521
Owner                   420
Director                402
Operations Manager      387
Truck Driver            387
Nurse                   326
Engineer                325
Sales Manager           304
manager                 301
Supervisor              270
Name: emp_title, dtype: int64

In [0]:
import numpy as np

In [0]:
top_titles = ['Teacher', 'Manager', 'Registered Nurse', 'Driver', 'RN',
              'Supervisor', 'Sales', 'Project Manager', 'General Manager',
              'Office Manager', 'Owner', 'Director', 'Operations Manager',
              'Truck Driver', 'Nurse', 'Engineer', 'Sales Manager', 'manager',
              'Supervisor', np.NaN ]
condition = df['emp_title'].isin(top_titles)
emp_title= df['emp_title'][condition]
def emp_title_2(title):
  if isinstance(title,str):
    return title.strip().title()
  else:
    return 'Other'
print(emp_title_2(item))

Other


In [0]:
df['emp_top_titles']=df['emp_title'].apply(emp_title_2)
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term',
       'term_int', 'term_double', 'loan_status_is_great', 'emp_top_titles'],
      dtype='object', length=148)

In [0]:
df[['emp_title', 'emp_top_titles']].sample(10)

,emp_title,emp_top_titles
28081,Nurse Practitioner,Nurse Practitioner
7183,NaN,Other
27871,Quality control,Quality Control
31955,Asst Manager,Asst Manager
58927,Public Safety,Public Safety
88545,SITE SUPERVISOR,Site Supervisor
80894,Project manager,Project Manager
104468,Teacher,Teacher
19610,NaN,Other
17100,Craftsman,Craftsman
